In [1]:
import tensorflow as tf
import jpeg_related_functions as jrf
import pickle as pkl
import jpeg_arch_test2 as jat


In [2]:
# Set parameters
jpeg_quality = 20
rho = 1.
alpha = 1.5
noi = 10
lmbda = 0.001
dtype = 'float64'

# Obtain quantization matrices from chosen quality factor
import PIL
import PIL.Image
import numpy as np
randImgPath = 'data/scratchwork/example/randImg.jpeg'
randimg = np.random.randint(0,256,size=(32,32,3))
encoded_jpeg = tf.image.encode_jpeg(randimg,quality = jpeg_quality)
tf.io.write_file(randImgPath,encoded_jpeg)
loadedRandImg = PIL.Image.open(randImgPath)
qY = np.asarray(loadedRandImg.quantization[0]).astype('uint8')
qUV = np.asarray(loadedRandImg.quantization[1]).astype('uint8')
qY = qY.astype(dtype)/255.
qUV = qUV.astype(dtype)/255.
import os
os.remove(randImgPath)

# setup smoothing layer
fftSz = (320,480)
smooth_jpeg = jat.Smooth_JPEG_Layer(rho,alpha,noi,qY,qUV,np.asarray(lmbda).astype('complex128'),fftSz,dtype=dtype)
# Loop through images
dataPath = 'data/original/TestA/'
datatype = 'train/'
filelist = os.listdir(dataPath + datatype)
imgs = []
for filename in filelist:
    loadedImg = PIL.Image.open(dataPath + datatype + filename)
    loadedImg = np.asarray(loadedImg).astype(dtype)/255.
    loadedImgShape = loadedImg.shape
    loadedImg = loadedImg[slice(0,loadedImgShape[0] - (loadedImgShape[0] % 8)),slice(0,loadedImgShape[1] - (loadedImgShape[1] % 8)),slice(None)]
    imgs.append(loadedImg)


In [3]:
def double(x):
    return (x,x)
dataset = tf.data.Dataset.from_tensor_slices(imgs)
dataset_with_output = dataset.map(double)
dataset_batch = dataset_with_output.batch(2)

inputs = tf.keras.layers.Input((320,480,3))
outputs,negC = smooth_jpeg(inputs)
model=tf.keras.Model(inputs,outputs)
model.compile(optimizer = tf.keras.optimizers.Adam(0.1),loss = tf.keras.losses.MSE)
model.fit(x=dataset_batch,epochs=3)


Epoch 1/3


AssertionError: in user code:

    /home/lrr/anaconda3/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /home/lrr/anaconda3/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/lrr/anaconda3/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/lrr/anaconda3/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/lrr/anaconda3/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/lrr/anaconda3/lib/python3.6/site-packages/tensorflow/python/keras/eng

In [4]:
model.save('saveModelTest.pb')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: saveModelTest.pb/assets


In [4]:
print(smooth_jpeg.xupdate.lmbda)

tf.Tensor((0.001+0j), shape=(), dtype=complex128)


In [4]:
print(model.trainable_variables)

[<tf.Variable 'Variable:0' shape=() dtype=float64, numpy=0.001>]
